# Chess Engine with TensorFlow

## Dataset

In [5]:
filename = "../../data/lichess_elite_2020-08.pgn"  
import os
# برای بررسی وجود فایل و خواندن محتوا  
if os.path.isfile(filename):  
    with open(filename, 'r') as file:  
        content = file.read()  # یا هرگونه پردازش دیگر که می‌خواهید انجام دهید  
else:  
    print(f"The file {filename} does not exist.")

In [2]:
len(filename)

36

In [3]:
# import os

# files = [file for file in os.listdir("../../data/lichess_elite_2020-08.pgn") if file.endswith(".pgn")]

In [ ]:
import chess
from chess import pgn

def load_pgn(file_path):
    games = []
    with open(file_path, 'r') as pgn_file:
        while True:
            game = pgn.read_game(pgn_file)
            if game is None:
                break
            games.append(game)
    return games

ModuleNotFoundError: No module named 'chess'

In [4]:
from tqdm import tqdm

LIMIT_OF_FILES = min(len(filename), 36)
games = []
i = 1
for file in tqdm(filename):
    games.extend(load_pgn(f"C:\\Users\\86034587\\Desktop\\chess-engine-main\\data\\lichess_elite_2020-08.pgn{filename}"))
    if (i >= LIMIT_OF_FILES):
        break
    i += 1

  0%|          | 0/36 [00:00<?, ?it/s]


NameError: name 'load_pgn' is not defined

In [5]:
len(games)

0

## Build & train a neural network

In [6]:
import numpy as np
from chess import Board
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Flatten, Dense
from tensorflow.keras.optimizers import Adam # type: ignore
import time

ModuleNotFoundError: No module named 'chess'

In [ ]:
def board_to_matrix(board: Board):
    matrix = np.zeros((8, 8, 12))
    piece_map = board.piece_map()
    for square, piece in piece_map.items():
        row, col = divmod(square, 8)
        piece_type = piece.piece_type - 1
        piece_color = 0 if piece.color else 6
        matrix[row, col, piece_type + piece_color] = 1
    return matrix


def create_input_for_nn(games):
    X = []
    y = []
    for game in games:
        board = game.board()
        for move in game.mainline_moves():
            X.append(board_to_matrix(board))
            y.append(move.uci())
            board.push(move)
    return X, y


def encode_moves(moves):
    move_to_int = {move: idx for idx, move in enumerate(set(moves))}
    return [move_to_int[move] for move in moves], move_to_int

In [ ]:
X, y = create_input_for_nn(games)
y, move_to_int = encode_moves(y)
y = to_categorical(y, num_classes=len(move_to_int))
X = np.array(X)

In [ ]:
model = Sequential([
    Conv2D(64, (3, 3), activation='relu', input_shape=(8, 8, 12)),
    Conv2D(128, (3, 3), activation='relu'),
    Flatten(),
    Dense(256, activation='relu'),
    Dense(len(move_to_int), activation='softmax')
])
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()
model.fit(X, y, epochs=50, validation_split=0.1, batch_size=64)
model.save("models/TF_50EPOCHS.keras")

## Predictions

In [ ]:
# Load the trained model
from tensorflow.keras.models import load_model
model = load_model("../../models/TF_50EPOCHS.keras")

In [ ]:
int_to_move = dict(zip(move_to_int.values(), move_to_int.keys()))


def predict_next_move(board):
    board_matrix = board_to_matrix(board).reshape(1, 8, 8, 12)
    predictions = model.predict(board_matrix)[0]
    legal_moves = list(board.legal_moves)
    legal_moves_uci = [move.uci() for move in legal_moves]
    sorted_indices = np.argsort(predictions)[::-1]
    for move_index in sorted_indices:
        move = int_to_move[move_index]
        if move in legal_moves_uci:
            return move
    return None

In [ ]:
# Create a chess board (start position)
board = Board()

In [ ]:
# Display the board before prediction
print("Board before prediction:")
print(board)

# Predict and make the move
next_move = predict_next_move(board)
board.push_uci(next_move)

# Display the board after prediction
print("\nPredicted move:", next_move)
print("Board after prediction:")
print(board)

In [ ]:
print(str(pgn.Game.from_board(board)))